<a href="https://colab.research.google.com/github/rishi-analytics/deep-learning/blob/main/Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Let's check the contents of your Drive to find the exact path
print("Checking directories...")

# Try different possible paths
possible_paths = [
    '/content/drive/MyDrive/DeepDrid',
    '/content/drive/MyDrive/DeepDRid',
    '/content/drive/MyDrive/DeepDrid/DeepDRid',
    '/content/drive/MyDrive/DeepDRid/DeepDRid'
]

for path in possible_paths:
    print(f"\nChecking path: {path}")
    try:
        contents = os.listdir(path)
        print("Contents found:", contents)
        print("This path exists!")
        correct_path = path
        break
    except:
        print("Path does not exist")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking directories...

Checking path: /content/drive/MyDrive/DeepDrid
Contents found: ['DeepDRiD']
This path exists!


In [ ]:
import os

# Verify files in the expected directories
csv_files = ['train.csv', 'val.csv', 'test.csv']
base_dir = '/content/drive/MyDrive/DeepDrid/DeepDRid'

print("Checking CSV files...")
for file in csv_files:
    file_path = os.path.join(base_dir, file)
    if os.path.exists(file_path):
        print(f"{file} exists at {file_path}")
    else:
        print(f"Error: {file} not found at {file_path}")

Checking CSV files...
Error: train.csv not found at /content/drive/MyDrive/DeepDrid/DeepDRid/train.csv
Error: val.csv not found at /content/drive/MyDrive/DeepDrid/DeepDRid/val.csv
Error: test.csv not found at /content/drive/MyDrive/DeepDrid/DeepDRid/test.csv


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Let's check the contents of your Drive to find the exact path
print("Checking directories...")

# Try different possible paths
possible_paths = [
    '/content/drive/MyDrive/DeepDrid',
    '/content/drive/MyDrive/DeepDRid',
    '/content/drive/MyDrive/DeepDrid/DeepDRid',
    '/content/drive/MyDrive/DeepDRid/DeepDRid'
]

for path in possible_paths:
    print(f"\nChecking path: {path}")
    try:
        contents = os.listdir(path)
        print("Contents found:", contents)
        print("This path exists!")
        correct_path = path
        break
    except:
        print("Path does not exist")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking directories...

Checking path: /content/drive/MyDrive/DeepDrid
Contents found: ['DeepDRiD']
This path exists!


In [ ]:
import torch
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/DeepDrid/DeepDRiD'

class DeepDRiDDataset(Dataset):
    def __init__(self, csv_file, base_dir, transform=None, mode='train'):
        self.data = pd.read_csv(csv_file)
        self.base_dir = base_dir
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.mode == 'test':
            img_path = os.path.join(self.base_dir, 'test', self.data.iloc[idx]['img_path'])
        elif self.mode == 'val':
            img_path = os.path.join(self.base_dir, 'val', self.data.iloc[idx]['img_path'])
        else:
            img_path = os.path.join(self.base_dir, 'train', self.data.iloc[idx]['img_path'])

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.mode != 'test':
            label = torch.tensor(self.data.iloc[idx]['patient_DR_Level'])
            return image, label
        else:
            return image, self.data.iloc[idx]['image_id']

def get_transforms(mode='train'):
    if mode == 'train':
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

def create_data_loaders(base_dir, batch_size=32):
    train_dataset = DeepDRiDDataset(
        csv_file=os.path.join(base_dir, 'train.csv'),
        base_dir=base_dir,
        transform=get_transforms('train'),
        mode='train'
    )

    val_dataset = DeepDRiDDataset(
        csv_file=os.path.join(base_dir, 'val.csv'),
        base_dir=base_dir,
        transform=get_transforms('val'),
        mode='val'
    )

    test_dataset = DeepDRiDDataset(
        csv_file=os.path.join(base_dir, 'test.csv'),
        base_dir=base_dir,
        transform=get_transforms('val'),
        mode='test'
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader, test_loader

# Print directory contents to verify
print("Contents of directory:", os.listdir(base_dir))

# Create data loaders
train_loader, val_loader, test_loader = create_data_loaders(base_dir, batch_size=32)

# Test the loader
for images, labels in train_loader:
    print("Batch shape:", images.shape)
    print("Labels shape:", labels.shape)
    break

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contents of directory: ['test', 'sample_submission.csv', 'test.csv', 'train.csv', 'val.csv', '.DS_Store', 'val', 'train']
Batch shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])


In [ ]:
import os
base_dir = '/content/drive/MyDrive/DeepDrid/DeepDRiD'

print("Checking directories and files...")
for subfolder in ['train', 'val', 'test']:
    path = os.path.join(base_dir, subfolder)
    print(f"Checking {subfolder} directory: {'Exists' if os.path.exists(path) else 'Missing'}")

csv_files = ['train.csv', 'val.csv', 'test.csv']
for file in csv_files:
    path = os.path.join(base_dir, file)
    print(f"Checking {file}: {'Exists' if os.path.exists(path) else 'Missing'}")

Checking directories and files...
Checking train directory: Exists
Checking val directory: Exists
Checking test directory: Exists
Checking train.csv: Exists
Checking val.csv: Exists
Checking test.csv: Exists


In [ ]:
import pandas as pd

# Check CSV content
base_dir = '/content/drive/MyDrive/DeepDrid/DeepDRiD'

for csv_file in ['train.csv', 'val.csv', 'test.csv']:
    path = os.path.join(base_dir, csv_file)
    print(f"\nContents of {csv_file}:")
    df = pd.read_csv(path)
    print(df.head())


Contents of train.csv:
   patient_id image_id    img_path  Overall quality  left_eye_DR_Level  \
0           1     1_l1  1/1_l1.jpg                0                0.0   
1           1     1_l2  1/1_l2.jpg                0                0.0   
2           1     1_r1  1/1_r1.jpg                0                NaN   
3           1     1_r2  1/1_r2.jpg                0                NaN   
4           2     2_l1  2/2_l1.jpg                0                2.0   

   right_eye_DR_Level  patient_DR_Level  Clarity  Field definition  Artifact  
0                 NaN                 0        8                 8         4  
1                 NaN                 0        8                 8         0  
2                 0.0                 0        8                 8         4  
3                 0.0                 0       10                 8         0  
4                 NaN                 2       10                 8         6  

Contents of val.csv:
   patient_id image_id        img_p

In [ ]:
train_loader, val_loader, test_loader = create_data_loaders(base_dir, batch_size=32)

# Test the train loader
for images, labels in train_loader:
    print("Batch shape:", images.shape)
    print("Labels shape:", labels.shape)
    break

Batch shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])


In [ ]:
import pandas as pd
import os

# Path to train.csv
train_csv_path = '/content/drive/MyDrive/DeepDrid/DeepDRiD/train.csv'

# Load the CSV and determine the number of classes
train_df = pd.read_csv(train_csv_path)
num_classes = train_df['patient_DR_Level'].nunique()  # Replace with your target column name
print(f"Number of classes: {num_classes}")

Number of classes: 5


In [ ]:
# Load a pretrained model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Adjust for your number of classes

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Load a pretrained model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Adjust for your number of classes

# Set up the training parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/10, Loss: 1.2281895828874487
Epoch 2/10, Loss: 0.9778132266119907
Epoch 3/10, Loss: 0.9343037542543913
Epoch 4/10, Loss: 0.8496744240585127
Epoch 5/10, Loss: 0.8506046392415699
Epoch 6/10, Loss: 0.8014564969037709
Epoch 7/10, Loss: 0.783104596953643
Epoch 8/10, Loss: 0.734376061903803
Epoch 9/10, Loss: 0.6665145833241312
Epoch 10/10, Loss: 0.6595332379403868


In [ ]:
# Saving the trained model after training
torch.save(model.state_dict(), "resnet18_deepdrid.pth")

In [ ]:
model.eval()  # Set the model to evaluation mode
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():  # No gradient calculation for validation
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = 100 * correct / total
print(f"Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy}%")

Validation Loss: 1.0208481275118315, Validation Accuracy: 62.5%


In [ ]:
from torchvision import transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Part B: Model Building and **Training**

In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('/content/drive/MyDrive/DeepDrid/DeepDRiD/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/DeepDrid/DeepDRiD/val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DeepDrid/DeepDRiD/test.csv')

# Inspect the datasets
print(f"Train Data Columns: \n{train_df.columns}")
print(f"Train Data Types: \n{train_df.dtypes}")
print(f"Train Data Summary: \n{train_df.describe()}")

# Check unique values in target column (e.g., patient_DR_Level)
print(f"Unique values in target column (patient_DR_Level): {train_df['patient_DR_Level'].unique()}")

Train Data Columns: 
Index(['patient_id', 'image_id', 'img_path', 'Overall quality',
       'left_eye_DR_Level', 'right_eye_DR_Level', 'patient_DR_Level',
       'Clarity', 'Field definition', 'Artifact'],
      dtype='object')
Train Data Types: 
patient_id              int64
image_id               object
img_path               object
Overall quality         int64
left_eye_DR_Level     float64
right_eye_DR_Level    float64
patient_DR_Level        int64
Clarity                 int64
Field definition        int64
Artifact                int64
dtype: object
Train Data Summary: 
        patient_id  Overall quality  left_eye_DR_Level  right_eye_DR_Level  \
count  1200.000000      1200.000000         600.000000          600.000000   
mean    151.330000         0.480000           1.303333            1.260000   
std      88.065983         0.499808           1.349453            1.349853   
min       1.000000         0.000000           0.000000            0.000000   
25%      75.750000         0

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import transforms

# Custom Dataset for loading images and labels from the CSV file
class DeepDridDataset(Dataset):
    def __init__(self, csv_file, root_dir, subdir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.subdir = subdir  # Add subdir to specify train/test/val
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.subdir, self.data.iloc[idx, 2])  # Image path including subdir
        print(f"Loading image: {img_name}")  # Debugging line
        if not os.path.exists(img_name):  # Check if file exists
            print(f"Warning: Image file {img_name} not found!")
            return None, None  # Skip this image if file not found

        image = Image.open(img_name)  # Open the image file
        label = self.data.iloc[idx, 6]  # Get the label for the image (patient_DR_Level)

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations (resize, normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18 expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Pre-trained ResNet mean/std
])

# Define dataset paths
train_dataset = DeepDridDataset(csv_file='/content/drive/MyDrive/DeepDrid/DeepDRiD/train.csv',
                                 root_dir='/content/drive/MyDrive/DeepDrid/DeepDRiD/',
                                 subdir='train',  # Specify subdirectory for train images
                                 transform=transform)

val_dataset = DeepDridDataset(csv_file='/content/drive/MyDrive/DeepDrid/DeepDRiD/val.csv',
                               root_dir='/content/drive/MyDrive/DeepDrid/DeepDRiD/',
                               subdir='val',  # Specify subdirectory for val images
                               transform=transform)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Checking a sample from train_loader (this is optional)
for images, labels in train_loader:
    if images is not None:  # Skip if there are missing files
        print(images.shape, labels.shape)
    break

Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/276/276_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/107/107_r1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/176/176_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/224/224_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/112/112_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/268/268_r1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/94/94_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/110/110_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/139/139_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/174/174_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/90/90_r1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/103/103_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/39/39_l2.jpg
Loadi

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match the number of classes (your specific task)
num_classes = 5  # Adjust this according to your dataset (for example, if you have 5 classes for DR Level)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Check if GPU is available, and move model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Print the model architecture
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Define the loss function (cross-entropy loss)
criterion = nn.CrossEntropyLoss()

# Define the optimizer (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Train phase
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            if inputs is None:  # Skip if there are any issues with image loading
                continue
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            # Track accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)
            total_preds += labels.size(0)

        # Calculate training loss and accuracy
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_preds.double() / total_preds
        print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        correct_preds = 0
        total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                if inputs is None:  # Skip if there are any issues with image loading
                    continue
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct_preds += torch.sum(preds == labels.data)
                total_preds += labels.size(0)

        val_acc = correct_preds.double() / total_preds
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Save the best model based on validation accuracy
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

    model.load_state_dict(best_model_wts)
    return model

# Train the model
num_epochs = 5
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs)

Streaming output truncated to the last 5000 lines.
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/357/357_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/357/357_r1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/357/357_r2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/358/358_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/358/358_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/358/358_r1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/358/358_r2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/359/359_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/359/359_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/359/359_r1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/359/359_r2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/val/360/360_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/De

In [ ]:
# Train the model for more epochs
num_epochs = 20  # Increase the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterate through the training set
    for images, labels in train_loader:
        images, labels = images.cuda(), labels.cuda()  # Move to GPU (if available)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct / total * 100
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / val_total * 100
    print(f"Validation Accuracy after Epoch {epoch + 1}: {val_accuracy:.2f}%")

Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/113/113_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/134/134_r2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/169/169_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/50/50_r1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/8/8_r2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/275/275_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/187/187_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/270/270_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/186/186_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/58/58_l2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/268/268_l1.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/274/274_r2.jpg
Loading image: /content/drive/MyDrive/DeepDrid/DeepDRiD/train/295/295_r1.jpg
Loading

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Save model weights
torch.save(model.state_dict(), '/content/drive/MyDrive/my_model.pth')